In [328]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import warnings
from pymystem3 import Mystem
from string import punctuation


warnings.filterwarnings("ignore")


In [331]:

russian_stopwords = stopwords.words("russian")

more_stop_words = ['это','также','данный', 'слово', 'который']
for word in more_stop_words:
    russian_stopwords.append(word)


mystem = Mystem()

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

### Get reports datasets

In [332]:
dataset_answered = pd.read_csv('Reports-examples.csv', error_bad_lines=False)
# dataset_answered = dataset_answered.dropna()


In [333]:
dataset_no_answers = pd.read_csv('Reports-no-answers.csv', error_bad_lines=False)
dataset_no_answers.drop([' spare column ', '№ обращения'],inplace=True, axis=1)


In [334]:

temp_df = dataset_no_answers 
# print(dataset_no_answers)

In [335]:
# dataset_answered
# dataset_no_answers
temp_df2 = temp_df
i = 0
for report in temp_df[' текст_обращения']:    
    # print(preprocess_text(report), '\n')
    temp_df2[' текст_обращения'][i] = preprocess_text(report)
    i += 1
temp_df2.head(10)


,текст_обращения
0,суть претензия мед работник находиться филиал ...
1,доводить ваш сведение горячий линия обращаться...
2,доводить ваш сведение горячий линия обращаться...
3,доводить ваш сведение горячий линия обращаться...
4,доводить ваш сведение горячий линия обращаться...
5,суть претензия – пациент обращаться пз 8 ...
6,доводить ваш сведение горячий линия обращаться...
7,доводить ваш сведение горячий линия обращаться...
8,доводить ваш сведение горячий линия обращаться...
9,претензия хотеть клиент регистратор пз коррект...


In [336]:
temp_df2.to_csv('clean_no_answered_dataset.csv')


### Get clear dataset

In [337]:
dataset_no_answers = pd.read_csv('clean_no_answered_dataset.csv')
# dataset_no_answers = temp_df2[' текст_обращения'].values
dataset_no_answers = dataset_no_answers.drop('Unnamed: 0', axis=1)

dataset_no_answers = dataset_no_answers[' текст_обращения'].values

# dataset_no_answers

### NMF

In [351]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 35
no_features = 40

tfidf_vectorizer = TfidfVectorizer(max_features=no_features, min_df=1, stop_words=russian_stopwords)
tfidf = tfidf_vectorizer.fit_transform(dataset_no_answers)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
tfidf_feature_names



['анализ',
 'время',
 'ждать',
 'забор',
 'звонок',
 'исследование',
 'клиент',
 'коронавирус',
 'кровь',
 'лаборатория',
 'линия',
 'мед',
 'медсестра',
 'мера',
 'мочь',
 'недовольный',
 'обратный',
 'обращаться',
 'обращение',
 'очередь',
 'пациент',
 'пациентка',
 'пз',
 'почта',
 'претензия',
 'принимать',
 'просить',
 'пцр',
 'разбираться',
 'ребенок',
 'регистратор',
 'результат',
 'свой',
 'связь',
 'сдавать',
 'сдача',
 'сестра',
 'сказать',
 'сотрудник',
 'хотеть']

In [352]:

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)


### LDA

In [353]:

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_features=no_features, stop_words=russian_stopwords)
tf = tf_vectorizer.fit_transform(dataset_no_answers)
tf_feature_names = tf_vectorizer.get_feature_names()
# tf_feature_names



In [354]:

# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.0, random_state=0).fit(tf)

In [355]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        s = "Topic %d: " % (topic_idx)
        s1 = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
        for word in s1:
            s += word
            s += ' '
        print(s)


no_top_words = 17
display_topics(nmf, tfidf_feature_names, no_top_words)
# display_topics(lda, tf_feature_names, no_top_words)


Topic 0: анализ хотеть претензия сдавать сдача клиент сказать пациент результат ждать забор звонок исследование недовольный коронавирус кровь время 
Topic 1: пациент разбираться пз претензия недовольный хотеть сотрудник результат сдача анализ обращаться пцр исследование обращение медсестра мера сдавать 
Topic 2: клиент хотеть претензия обратный анализ пациентка просить сказать сдавать кровь мера медсестра мед линия лаборатория коронавирус недовольный 
Topic 3: пациентка пз претензия хотеть линия недовольный свой принимать анализ регистратор мочь время ждать забор звонок исследование клиент 
Topic 4: медсестра ждать обращаться пациент анализ звонок исследование клиент коронавирус кровь обращение линия мед забор мера мочь недовольный 
Topic 5: претензия хотеть обращение обратный недовольный мочь мера медсестра мед линия лаборатория кровь коронавирус клиент исследование звонок забор 
Topic 6: сотрудник пз пациент ждать просить хотеть мед недовольный мочь мера медсестра линия обращаться ла

### Test movie-review dataset

### расстояния махеттенское, ливинштейна
исправление опечаток???